In [1]:
// Declare the values for your Azure SQL DB
val jdbcUsername = "USERNAME"
val jdbcPassword = "PASSWORD"
val jdbcHostname = "URL.database.windows.net"
val jdbcPort = 1433
val jdbcDatabase = "ContosoRetail"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1517881491727_0004,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
jdbcDatabase: String = ContosoRetail

In [2]:
import java.util.Properties

val jdbc_url = s"jdbc:sqlserver://${jdbcHostname}:${jdbcPort};database=${jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=60;"
val connectionProperties = new Properties()
connectionProperties.put("user",s"${jdbcUsername}")
connectionProperties.put("password",s"${jdbcPassword}")

res4: Object = null

In [3]:
val sqlTableDF = spark.read.jdbc(jdbc_url, "ContosoRetail.dbo.FactInventory", connectionProperties)

sqlTableDF: org.apache.spark.sql.DataFrame = [InventoryKey: int, DateKey: timestamp ... 14 more fields]

In [4]:
sqlTableDF.printSchema

root
 |-- InventoryKey: integer (nullable = false)
 |-- DateKey: timestamp (nullable = false)
 |-- StoreKey: integer (nullable = false)
 |-- ProductKey: integer (nullable = false)
 |-- CurrencyKey: integer (nullable = false)
 |-- OnHandQuantity: integer (nullable = false)
 |-- OnOrderQuantity: integer (nullable = false)
 |-- SafetyStockQuantity: integer (nullable = true)
 |-- UnitCost: decimal(19,4) (nullable = false)
 |-- DaysInStock: integer (nullable = true)
 |-- MinDayInStock: integer (nullable = true)
 |-- MaxDayInStock: integer (nullable = true)
 |-- Aging: integer (nullable = true)
 |-- ETLLoadID: integer (nullable = true)
 |-- LoadDate: timestamp (nullable = true)
 |-- UpdateDate: timestamp (nullable = true)

In [7]:
sqlTableDF.show(1)

An error was encountered:
Session 0 did not reach idle status in time. Current status is busy.


In [8]:
sqlTableDF.select("OnHandQuantity","UnitCost").show(1)

An error was encountered:
Session 0 did not reach idle status in time. Current status is busy.


In [10]:
// extracts schema data in the .CSV and loads the data from .CSV in a dataframe

val userSchema = spark.read.option("header", "true").csv("wasbs:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv").schema
val readDf = spark.read.format("csv").schema(userSchema).load("wasbs:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv")

readDf: org.apache.spark.sql.DataFrame = [Date: string, Time: string ... 5 more fields]

In [11]:
// readDf creates a temp table, then use temp table to create a hive table

readDf.createOrReplaceTempView("temphvactable")
spark.sql("create table hvactable_hive as select * from temphvactable")

res8: org.apache.spark.sql.DataFrame = []

In [12]:
// use the hive table to create a table in Azure SQL DB
spark.table("hvactable_hive").write.jdbc(jdbc_url, "hvactable", connectionProperties)

In [13]:
// packages for streaming into a hvac table

import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming._
import java.sql.{Connection,DriverManager,ResultSet}

import java.sql.{Connection, DriverManager, ResultSet}

In [17]:
// Stream data from .CSV into the hvac table
// Get the schema of the data to be streamed
// Create a streaming dataframe using that schema.

val userSchema = spark.read.option("header", "true").csv("wasbs:///HdiSamples/HdiSamples/SensorSampleData/hvac/HVAC.csv").schema
val readStreamDf = spark.readStream.schema(userSchema).csv("wasbs:///HdiSamples/HdiSamples/SensorSampleData/hvac/") 
readStreamDf.printSchema

root
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- TargetTemp: string (nullable = true)
 |-- ActualTemp: string (nullable = true)
 |-- System: string (nullable = true)
 |-- SystemAge: string (nullable = true)
 |-- BuildingID: string (nullable = true)

In [19]:
val WriteToSQLQuery = readStreamDf.writeStream.foreach(new ForeachWriter[Row] {
    var connection:java.sql.Connection = _
    var statement:java.sql.Statement = _
    
    val jdbcUserName = "USERNAME"
    val jdbcPassword = "PASSWORD"
    val jdbcHostname = "URL.database.windows.net"
    val jdbcPort = 1433
    val jdbcDatabase = "ContosoRetail"
    val driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    val jdbc_url = s"jdbc:sqlserver://${jdbcHostname}:${jdbcPort};database=${jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
    
    def open(partitionId: Long, version: Long):Boolean = {
        Class.forName(driver)
        connection = DriverManager.getConnection(jdbc_url, jdbcUsername, jdbcPassword)
        statement = connection.createStatement
        true
    }
    
    def process(value: Row): Unit = {
        val Date = value(0)
        val Time = value(1)
        val TargetTemp = value(2)
        val ActualTemp = value(3)
        val System = value(4)
        val SystemAge = value(5)
        val BuildingID = value(6)
        
        val valueStr = "'" + Date + "'" + Time + "'" + TargetTemp + "'" + ActualTemp + "'" + System + "'" + SystemAge + "'" + BuildingID + "'"
        statement.execute("INSERT INTO " + "dbo.hvactable" + " VALUES (" + valueStr + ")")
        }
    
    def close(errorOrNull: Throwable): Unit = {
        connection.close
    }
})

var steamingQuery = WriteToSQLQuery.start()

steamingQuery: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@4346c691